In [4]:
#Import required Libraries
import pandas as pd
import requests, os
from sqlalchemy import create_engine, exc
import pandas as pd
from configparser import ConfigParser
#Db Connect Configuration
import psycopg2

In [79]:
#Get data file from downloaded path
#source: https://databank.worldbank.org/data/download/GEP_CSV.zip
GDP_data_csv = pd.read_csv("C:/Users/dganesan26/OneDrive - DXC Production/desktop/GEPData.csv") 

In [6]:
#This function is available on public repository- Used to read the Json link from python
def data_from_json(endpoint):
    response = requests.get(endpoint,timeout=20)
    response = response.json()
    return_lst = response[1]

    while response[0]['page']<response[0]['pages']:
        endpoint_pg = f"{endpoint}&page={int(response[0]['page'])+1}"
        response = requests.get(endpoint_pg,timeout=20)
        response = response.json()
        return_lst = return_lst + response[1]
    return pd.json_normalize(return_lst)

In [12]:
#Get data from Json
#Json data api source- "http://api.worldbank.org/v2/country?format=json"
Get_Data_Json = data_from_json("http://api.worldbank.org/v2/country?format=json")

In [13]:
#Data Iteration-Remove special characters & spaces
Get_Data_Json.columns = Get_Data_Json.columns.str.replace(".", "", regex=True)
Get_Data_Json.columns = Get_Data_Json.columns.str.replace(' ','')
#Get_Data_Json.head()

In [14]:
# write the gdp data scrapped from JSON to the local folder 
Get_Data_Json.to_csv('C:/Users/dganesan26/OneDrive - DXC Production/desktop/GDP_Data_modified.csv')   

In [59]:
# Get Region and Country from JSON data
GDP_data_01 = Get_Data_Json.loc[Get_Data_Json['regionid']!='NA']
GDP_data_02 = Get_Data_Json.loc[Get_Data_Json['regionid']=='NA']

In [36]:
#Checkpoint-connect to DB
#conn = psycopg2.connect("dbname=postgres user=***** password=*****")

In [174]:
#This function is available on public repository- Used to connect DB from Python
#Source- https://www.postgresqltutorial.com/postgresql-python/connect/
#!/usr/bin/python
from configparser import ConfigParser
def postgre_db(filename='/config.ini', section='postgresql'):
  
    parser = ConfigParser()

    # read config file
    parser.read(filename)

    # get section, default to postgresql
    params = {}
    if parser.has_section(section):
        params_ext = parser.items(section)
        for param in params_ext:
            params[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
        
    # intialise the engine variable 
    engine = create_engine

    try:
       
        connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
                    params['user'],
                    params['password'],
                    params['host'],
                    params['database']
                    )
        engine = create_engine(connect)
    except (exc.SQLAlchemyError) as error:
        print(error)

    return engine

In [63]:
def SQL_WRITE(df,engine,tableName):
    df.to_sql(
        tableName, 
        con=engine, 
        index=False, 
        if_exists='replace'
    )

In [64]:
# Create a postgres db engine using the properties in the config.ini file 
engine = postgre_db()

In [80]:
# Insert the data in to postgres database.
#Country & Region Details written seperatly
SQL_WRITE(GDP_data_01,engine,'GDPDATA_CNTY')
SQL_WRITE(GDP_data_02,engine,'GDPDATA_REGION')
#Full details from CSV file dumped to details table
SQL_WRITE(GDP_data_csv,engine,'GDPDATA_DETAILS')

In [185]:
#Data Analysis- Build  SQL for each of the analysis
#1. List countries with income level of "Upper middle income"
sql_text = """select "adminregionvalue" as "Region", "name" as "Country","incomeLevelvalue" as "Income Level" from public."GDPDATA_CNTY"
where "incomeLevelid" = 'UMC'"""

Analysis1 = pd.read_sql(sql_text,engine)
Analysis1

,Region,Country,Income Level
0,Europe & Central Asia (excluding high income),Albania,Upper middle income
1,Latin America & Caribbean (excluding high income),Argentina,Upper middle income
2,Europe & Central Asia (excluding high income),Armenia,Upper middle income
3,East Asia & Pacific (excluding high income),American Samoa,Upper middle income
4,Europe & Central Asia (excluding high income),Azerbaijan,Upper middle income
5,Europe & Central Asia (excluding high income),Bulgaria,Upper middle income
6,Europe & Central Asia (excluding high income),Bosnia and Herzegovina,Upper middle income
7,Europe & Central Asia (excluding high income),Belarus,Upper middle income
8,Latin America & Caribbean (excluding high income),Brazil,Upper middle income
9,Sub-Saharan Africa (excluding high income),Botswana,Upper middle income


In [71]:
#2. countries with income level of "Low income" per region.
sql_text = """select "regionid", "adminregionvalue" as "Region", "name" as "Country", 
"incomeLevelvalue" as "Income Level" from public."GDPDATA_CNTY" where "incomeLevelid" = 'LIC'
group by (regionid, adminregionvalue,"name", "incomeLevelvalue")
order by regionid """
Analysis2 = pd.read_sql(sql_text,engine)
Analysis2

,regionid,Region,Country,Income Level
0,EAS,East Asia & Pacific (excluding high income),"Korea, Dem. People's Rep.",Low income
1,MEA,Middle East & North Africa (excluding high inc...,Syrian Arab Republic,Low income
2,MEA,Middle East & North Africa (excluding high inc...,"Yemen, Rep.",Low income
3,SAS,South Asia,Afghanistan,Low income
4,SSF,Sub-Saharan Africa (excluding high income),Burkina Faso,Low income
5,SSF,Sub-Saharan Africa (excluding high income),Burundi,Low income
6,SSF,Sub-Saharan Africa (excluding high income),Central African Republic,Low income
7,SSF,Sub-Saharan Africa (excluding high income),Chad,Low income
8,SSF,Sub-Saharan Africa (excluding high income),"Congo, Dem. Rep.",Low income
9,SSF,Sub-Saharan Africa (excluding high income),Eritrea,Low income


In [186]:
#3.Region with the highest proportion of "High income" countries.
sql_text = """select xx.region_rank as "Position",xx.name as "Region",xx.noofcountries "High Income Countries-Total" 
from(select yy.regionid,yy.noofcountries, RANK() OVER( ORDER BY yy.noofcountries DESC) as "region_rank" ,zz.name 
from (select regionid,count(name) as noofcountries
from public."GDPDATA_CNTY" where "incomeLevelid" = 'HIC'group by (regionid)) yy
LEFT JOIN public."GDPDATA_REGION" as zz on zz.id = yy.regionid ) xx
where xx.region_rank = 1; """
Analysis3 = pd.read_sql(sql_text,engine)
Analysis3

,Position,Region,High Income Countries-Total
0,1,Europe & Central Asia,37


In [114]:
# 4. cumulative/running value of GDP per region ordered by income from lowest to highest and country
SQL_TEXT = """select AA.regionvalue as "Region", AA.final2021 as "Cumulative"
from (select XX.regionid, XX.regionvalue, 
ROUND (sum(BB."2018")::numeric,2) as final2018,ROUND (sum(BB."2019")::numeric,2) as final2019, 
ROUND (sum(BB."2020")::numeric,2) as final2020,ROUND (sum(BB."2021")::numeric,2) as final2021,ROUND (sum(BB."2022")::numeric,2) as final2022
from(select "Country Code","2018","2019","2020","2021","2022"
from public."GDPDATA_DETAILS" where "Country Code" <> ALL (array['AME','EAA','EMD','E19','ECH','LAP','MNH','SAP','SSP','WLT'])) BB  
LEFT JOIN public."GDPDATA_CNTY" as XX on XX.id = BB."Country Code" GROUP BY (XX.regionid,XX.regionvalue)) AA
GROUP BY (AA.regionvalue,AA.final2021)
ORDER BY (final2021) asc"""
Analysis4 = pd.read_sql(SQL_TEXT,engine)
Analysis4

,Region,Cumulative
0,North America,3.5
1,South Asia,22.7
2,Middle East & North Africa,27.0
3,East Asia & Pacific,65.7
4,Europe & Central Asia,78.2
5,Latin America & Caribbean,96.3
6,Sub-Saharan Africa,134.3


In [187]:
# 5. % difference in value of GDP year-on-year per country
sql_text = """select "Country Name",(("2019"-"2018")*100)/(CASE WHEN "2018"=0 THEN 1 ELSE "2018" END) as "Change_2019",
(("2020"-"2019")*100)/(CASE WHEN "2019"=0 THEN 1 ELSE "2019" END) as "Change_2020",
(("2021"-"2020")*100)/(CASE WHEN "2020"=0 THEN 1 ELSE "2020" END) as "Change_2021",
(("2022"-"2021")*100)/(CASE WHEN "2021"=0 THEN 1 ELSE "2021" END) as "Change_2022" 
from (select "Country Name","Country Code","Indicator Name","Indicator Code","2018","2019","2020","2021","2022"
from public."GDPDATA_DETAILS" 
where "Country Code" <> ALL (array['AME','EAA','EMD','E19','ECH','LAP','MNH','SAP','SSP','WLT'])) AA"""
Analysis5 = pd.read_sql(sql_text,engine)
Analysis5

,Country Name,Change_2019,Change_2020,Change_2021,Change_2022
0,Afghanistan,225.000000,-241.025641,-145.454545,32.000000
1,Albania,-46.341463,-404.545455,-176.119403,-13.725490
2,Algeria,-33.333333,-912.500000,-158.461538,-44.736842
3,Angola,-55.000000,344.444444,-122.500000,288.888889
4,Argentina,-19.230769,404.761905,-146.226415,-61.224490
...,...,...,...,...,...
131,Vietnam,-1.408451,-60.000000,139.285714,-2.985075
132,West Bank and Gaza,16.666667,-664.285714,-129.113924,4.347826
133,Zambia,-60.000000,-421.428571,-142.222222,78.947368
134,Zimbabwe,-268.750000,23.456790,-129.000000,6.896552


In [192]:
#6 countries with lowest GDP per regionquery =
sql_text = """select QUERY.* 
from(select XX.regionid, XX.regionvalue, XX.id as "country_id",YY. "Country Name", YY."2021" as "Gdp2021", 
RANK() OVER(PARTITION BY XX.regionid ORDER BY YY."2021" ASC) as "Rank_Country" 
from (select "Country Name","Country Code", "Indicator Name","Indicator Code","2018","2019","2020","2021","2022"
from public."GDPDATA_DETAILS" 
where "Country Code"<> ALL (array['AME','EAA','EMD','E19','ECH','LAP','MNH','SAP','SSP','WLT'])) YY  
LEFT JOIN public."GDPDATA_CNTY" as XX on XX.id = YY."Country Code" ) QUERY
WHERE QUERY."Rank_Country" < 4
ORDER BY QUERY.regionid,QUERY."Rank_Country"  """
Analysis6 = pd.read_sql(sql_text,engine)
Analysis6

,regionid,regionvalue,country_id,Country Name,Gdp2021,Rank_Country
0,EAS,East Asia & Pacific,MMR,Myanmar,2.0,1
1,EAS,East Asia & Pacific,JPN,Japan,2.5,2
2,EAS,East Asia & Pacific,FJI,Fiji,2.6,3
3,ECS,Europe & Central Asia,BLR,Belarus,-2.7,1
4,ECS,Europe & Central Asia,AZE,Azerbaijan,1.9,2
5,ECS,Europe & Central Asia,KAZ,Kazakhstan,2.5,3
6,LCN,Latin America & Caribbean,SUR,Suriname,-1.9,1
7,LCN,Latin America & Caribbean,NIC,Nicaragua,-0.9,2
8,LCN,Latin America & Caribbean,VCT,St. Vincent and the Grenadines,0.0,3
9,MEA,Middle East & North Africa,LBN,Lebanon,-13.2,1
